<a href="https://colab.research.google.com/github/konductor000/tg-GPT-2_bot/blob/main/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install tensorflow-gpu==1.12.0
!pip3 install pyTelegramBotAPI


     |████████████████████████████████| 51kB 7.1MB/s 
  Using cached https://files.pythonhosted.org/packages/07/53/8d32ce9471c18f8d99028b7cef2e5b39ea8765bd7ef250ca05b490880971/tensorboard-1.12.2-py3-none-any.whl
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.12.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1


In [2]:
# Use Tensorflow 1
%tensorflow_version 1.x

# Select GPT-2 model: 117M, 124M, 355M, 774M, 1558M

model_name = '1558M'     # '117M' - smallest model. Set '1558M' for biggest 1.5B model

!git clone https://github.com/openai/gpt-2

%cd gpt-2

!pip3 install -r requirements.txt

# Download GPT-2 (selected model, for example 117M)
!python3 download_model.py $model_name



TensorFlow 1.x selected.
fatal: destination path 'gpt-2' already exists and is not an empty directory.
/content/gpt-2
Fetching checkpoint: 1.00kit [00:00, 1.09Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 4.62Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.18Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 6.23Git [03:58, 26.1Mit/s]                                 
Fetching model.ckpt.index: 21.0kit [00:00, 727kit/s]                                                
Fetching model.ckpt.meta: 1.84Mit [00:00, 4.69Mit/s]                                                
Fetching vocab.bpe: 457kit [00:00, 3.20Mit/s]                                                       


In [3]:
%cd src

/content/gpt-2/src


In [4]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder
import telebot


In [5]:
class GPT():


  def __init__(
      self,
      model_name,
      seed,
      length,
      temperature,
      top_k,
      models_dir
  ):
      models_dir = os.path.expanduser(os.path.expandvars(models_dir))


      self.enc = encoder.get_encoder(model_name, models_dir)
      hparams = model.default_hparams()
      with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
          hparams.override_from_dict(json.load(f))

      if length is None:
          length = hparams.n_ctx // 2
      elif length > hparams.n_ctx:
          raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

      g = tf.Graph() 
      with g.as_default():
      
        self.context = tf.placeholder(tf.int32, [1, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        self.output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=self.context,
            batch_size=1,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
      
      self.sess = tf.compat.v1.Session(graph=g) 
      saver.restore(self.sess, ckpt)


  def print_message(self, raw_text):
    context_tokens = self.enc.encode(raw_text)
    
    out = self.sess.run(self.output, feed_dict={
        self.context: [context_tokens for _ in range(1)]
    })[:, len(context_tokens):]
    for i in range(1):
        text = self.enc.decode(out[i])
        return text


In [ ]:
gpt = GPT(
    model_name,
    None,
    100,
    1,
    40,
    '/content/gpt-2/models'
)

Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use `tf.random.categorical` instead.


Instructions for updating:
Use `tf.random.categorical` instead.


INFO:tensorflow:Restoring parameters from /content/gpt-2/models/1558M/model.ckpt


INFO:tensorflow:Restoring parameters from /content/gpt-2/models/1558M/model.ckpt


In [ ]:


bot = telebot.TeleBot("1602318304:AAG0YvOX8MhX7nEWqmJjyVpq5EdQhgTnVLg")

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, f'Hi, {message.from_user.first_name}! I am a GPT-2 bot. Enter some text and you will receive a continue of your text.')


@bot.message_handler(content_types=['text'])
def send_text(message):
    text = message.text
    bot.send_message(message.chat.id, text + gpt.print_message(text))

bot.polling(none_stop=True)